# NASA - LURA Project 2022-2023

Python code for the paper titled "Exoplanet Atmosphere Characterization via Transit Spectra Classification"

Cite: Fortela, D.L.; Travis, A.; Mikolajczyk, A.; Sharp, W. Exoplanet Atmosphere Characterization via Transit Spectra Classification. Preprints 2023, 2023081385. https://doi.org/10.20944/preprints202308.1385.v1



## PLATON - Python Package for Exoplanet Atmospheric Spectra Simulation

URL: https://platon.readthedocs.io/en/latest/quickstart.html

NOTE: make sure to install the PLATON module before running these codes. See link above for the instructons and examples of running the PLATON 5.3 module.

In [1]:
import pandas as pd
import numpy as np

from platon.transit_depth_calculator import TransitDepthCalculator
from platon.constants import M_jup, R_jup, R_sun
import matplotlib.pyplot as plt


## Now specify the atmospheric abundance of chemicals

In [3]:
from platon.abundance_getter import AbundanceGetter


from platon.transit_depth_calculator import TransitDepthCalculator
from platon.constants import M_jup, R_jup, R_sun


# All inputs and outputs for PLATON are in SI

Rs = 1.16 * R_sun
Mp = 0.73 * M_jup
Rp = 1.40 * R_jup
T = 1200


# specify adundace
#abundances = AbundanceGetter.from_file("~/platon/examples/custom_abundances/abund_C2H2.dat")
getter = AbundanceGetter()

# Solar logZ and C/O ratio. Modify as required.
abundances = getter.get(0, 0.53)

# Zero out CO.  (Note that if CO is a major component, you should probably
# renormalize the abundances of other species so that they add up to 1.)
abundances["CO"] *= 0


# create the array of CO2 and CH4 levels: [co2, ch4, o2] as tuples
CO2_lvls_low = np.random.uniform(low=0.0, high=9.9e-5, size=10)
CO2_lvls_high = np.random.uniform(low=10e-5, high=10e-4, size=10)

O2_lvls_low = np.random.uniform(low=0.0, high=9.9e-5, size=10)
O2_lvls_high = np.random.uniform(low=10e-5, high=10e-4, size=10)

N2_lvls_low = np.random.uniform(low=0.0, high=9.9e-5, size=10)
N2_lvls_high = np.random.uniform(low=10e-5, high=10e-4, size=10)

# combine the low and high levels of CO2, CH4, and O2 in all possible ways to create the labels
CO2_lvls_low = np.reshape(CO2_lvls_low, (CO2_lvls_low.shape[0],1))
CO2_lvls_high = np.reshape(CO2_lvls_high, (CO2_lvls_high.shape[0],1))

O2_lvls_low = np.reshape(O2_lvls_low, (O2_lvls_low.shape[0],1))
O2_lvls_high = np.reshape(O2_lvls_high, (O2_lvls_high.shape[0],1))

N2_lvls_low = np.reshape(N2_lvls_low, (N2_lvls_low.shape[0],1))
N2_lvls_high = np.reshape(N2_lvls_high, (N2_lvls_high.shape[0],1))


# setup level-combinations of ch2, ch4, o2 to create categories of stecra/labels: 2^3 = 8 labels
level_1 = np.concatenate((CO2_lvls_low, O2_lvls_low, N2_lvls_low), axis=1)
level_2 = np.concatenate((CO2_lvls_high, O2_lvls_low, N2_lvls_low), axis=1)
level_3 = np.concatenate((CO2_lvls_low, O2_lvls_high, N2_lvls_low), axis=1)
level_4 = np.concatenate((CO2_lvls_high, O2_lvls_high, N2_lvls_low), axis=1)
level_5 = np.concatenate((CO2_lvls_low, O2_lvls_low, N2_lvls_high), axis=1)
level_6 = np.concatenate((CO2_lvls_high, O2_lvls_low, N2_lvls_high), axis=1)
level_7 = np.concatenate((CO2_lvls_low, O2_lvls_high, N2_lvls_high), axis=1)
level_8 = np.concatenate((CO2_lvls_high, O2_lvls_high, N2_lvls_high), axis=1)



list_levels = np.concatenate((level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8), axis=0)


# convert numpy array into tuples
levels_tuples = tuple(map(tuple, list_levels))


# initialize empty spectra array
arr = []
loop_idx = 1


for co2, o2, n2 in levels_tuples:
    
    #print("Loop" + str(loop_idx))
    
    loop_idx += 1
    
    # Set CH4 abundance to a constant throughout the atmosphere
    abundances["CO2"] *= 0
    abundances["CO2"] += co2


    # Set CO2 abundance to a constant throughout the atmosphere
    abundances["O2"] *= 0
    abundances["O2"] += o2

    
    # Set o2 abundance to a constant throughout the atmosphere
    abundances["N2"] *= 0
    abundances["N2"] += n2
    
    
    # The initializer loads all data files.  Create a TransitDepthCalculator
    # object and hold on to it
    calculator = TransitDepthCalculator(method="xsec") #"ktables" for correlated k


    # compute_depths is fast once data files are loaded
    wavelengths, depths, info_dict = calculator.compute_depths(Rs, 
                                                               Mp, 
                                                               Rp, 
                                                               T, 
                                                               logZ=None, 
                                                               CO_ratio=None, 
                                                               custom_abundances=abundances, 
                                                               full_output=True)



    # save simulated data
    wav_um = np.reshape(1e6*wavelengths, (wavelengths.shape[0], 1))
    depth = np.reshape(depths, (depths.shape[0], 1))
    
    arr.append(depth)

    


In [5]:
# add wavelength column
arr2 = np.array(arr)
arr2 = np.reshape(arr2, (list_levels.shape[0],arr2.shape[1]))
wav_um2 = wav_um.transpose()
wav_um3 = np.insert(wav_um2, 0, [0,0,0])
wav_um4 = np.reshape(wav_um3, (1,wav_um3.shape[0]))

spec_arr = np.concatenate((np.reshape(list_levels, (list_levels.shape[0],3)), arr2),axis=1)
spec_arr2 = np.concatenate((wav_um4, spec_arr),axis=0)

# convert array into dataframe
DF = pd.DataFrame(spec_arr2)

# save the dataframe as a csv file
filename = "3-Sample_data_Set1_O2varyCO2varyCH4_looped_workcsv"
DF.to_csv(filename, header=False, index=False)
